In [1]:
!pip install requests beautifulsoup4 pandas

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
ticker = 'MSFT'

In [5]:
def fetch_finviz_news(stock_symbol):
    url = f"https://finviz.com/quote.ashx?t={stock_symbol}"
    headers = {'User-Agent': 'Mozilla/5.0'}  # Some websites require a user-agent header
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        headlines = []

        # Attempt to locate the news table by its class. Update if the class name has changed.
        news_table = soup.find('table', class_='fullview-news-outer')
        if news_table:
            for row in news_table.findAll('tr'):
                try:
                    text = row.a.text.strip() if row.a else 'No headline available'
                    link = row.a['href'] if row.a else 'No link available'
                    date = row.td.text.strip().split('\n')[0] if row.td else 'No date available'
                    headlines.append({'headline': text, 'link': link, 'date': date})
                except AttributeError:
                    continue  # Skip rows where expected elements are missing
        else:
            return "News table not found."

        return headlines
    else:
        return "Failed to retrieve news."

In [11]:
# Fetch news for AAPL
news = fetch_finviz_news(ticker)

# Convert to DataFrame for easier viewing
news_df = pd.DataFrame(news)

# Display the first few rows
print(news_df.head())

                                            headline  \
0  Magnificent Seven Stocks To Buy And Watch: Nvi...   
1  Today's Dow Jones Stocks To Watch: Apple Stock...   
2  Stock Rally Keeps Going With Dow, S&P 500 At N...   
3  The Hot Buzzword For Cybersecurity Stocks Isn'...   
4  Justice Department files antitrust suit agains...   

                                                link           date  
0  https://finance.yahoo.com/m/4205eaa9-f620-3a0b...  Today 11:30AM  
1  https://finance.yahoo.com/m/65b53896-faf4-3a06...        11:29AM  
2  https://finance.yahoo.com/m/460b1bf5-53d3-3e09...        11:06AM  
3  https://finance.yahoo.com/m/d9f19484-6677-31c9...        11:05AM  
4  https://finance.yahoo.com/news/justice-departm...        10:55AM  


In [12]:
news_df.to_csv(f'{ticker}_stock_news.csv', index=False)
print(f"{ticker} stock news saved to aapl_stock_news.csv.")

MSFT stock news saved to aapl_stock_news.csv.


In [16]:
import os

folder_path = f'/IGP Notebooks/Prepared Data/Stock News/{ticker}/'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)


In [17]:
from datetime import datetime

# Current date and time, formatted as YYYY-MM-DD_HHMMSS
now = datetime.now().strftime("%Y-%m-%d_%H%M%S")

In [18]:
file_path = os.path.join(folder_path, f'{ticker}_stock_news_{now}.csv')
news_df.to_csv(file_path, index=False)
print(f"File saved to {file_path}")


File saved to /IGP Notebooks/Prepared Data/Stock News/MSFT/MSFT_stock_news_2024-03-21_155407.csv
